In [1]:

import os
import cv2 as cv # type: ignore
import numpy as np # type: ignore
from time import time
from utils.functions import *
from multiprocessing import Process, Lock, Manager

# 读入视频的路径 
video1_path = '../data/testolabc1.avi'
video2_path = '../data/testolabc2.avi'
video3_path = '../data/testolabc3.avi'
video4_path = '../data/testolabc4.avi'

# 计算单应性矩阵
H1, H2, H3 = compute_homographies(video1_path, video2_path, video3_path, video4_path)


In [2]:

# 定义多线程处理的函数
def process_frames(start_frame, end_frame, lock, output_dir, start_time, frame_times, total_frames, process_id):
    video1_path = '../data/testolabc1.avi'
    video2_path = '../data/testolabc2.avi'
    video3_path = '../data/testolabc3.avi'
    video4_path = '../data/testolabc4.avi'

    for frame_number in range(start_frame, end_frame):
        frame_start_time = time()
        frame1, frame2 = read_frames_from_videos(video1_path, video2_path, frame_number)
        frame3, frame4 = read_frames_from_videos(video3_path, video4_path, frame_number)

        frame1, frame2 = undistort_and_rotate(frame1, dist_coeffs=np.array([-0.0733, 0.0833, 0, 0], dtype=np.float32), angle=-2.125), undistort_and_rotate(frame2, dist_coeffs=np.array([-0.0733, 0.0833, 0, 0], dtype=np.float32))
        frame1, frame2 = resize_to_equal_width(frame1, frame2)
        frame3, frame4 = undistort_and_rotate(frame3, dist_coeffs=np.array([-0.0733, 0.0833, 0, 0], dtype=np.float32), angle=-2.125), undistort_and_rotate(frame4, dist_coeffs=np.array([-0.0733, 0.0833, 0, 0], dtype=np.float32))
        frame3, frame4 = resize_to_equal_width(frame3, frame4)

        sti1, sti2 = stitch_images_with_blending(frame1, frame2, H1), stitch_images_with_blending(frame3, frame4, H2)
        result = stitch_images_with_blending(sti1, sti2, H3)

        frame_elapsed_time = time() - frame_start_time
        with lock:
            frame_times[process_id].append(frame_elapsed_time)
            total_elapsed_time = sum([sum(times) for times in frame_times.values()])
            total_processed_frames = sum([len(times) for times in frame_times.values()])
            avg_frame_time = total_elapsed_time / total_processed_frames if total_processed_frames > 0 else 0
            frame_rate = num_processes / avg_frame_time if avg_frame_time > 0 else 0

        text = f"FPS: {frame_rate:.2f} Time: {frame_elapsed_time:.2f}"
        cv.putText(result, text, (10, 30), cv.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

        with lock:
            output_path = os.path.join(output_dir, f'{frame_number}.jpg')
            cv.imwrite(output_path, result)

if __name__ == "__main__":
    start_time = time()
    output_dir = 'output'
    os.makedirs(output_dir, exist_ok=True)

    num_processes = 16  # 设置进程数量
    total_frames = 1800  # 总帧数
    frames_per_process = total_frames // num_processes
    lock = Lock()
    manager = Manager()
    frame_times = manager.dict({i: manager.list() for i in range(num_processes)})

    processes = []

    # 启动多个进程
    for i in range(num_processes):
        start_frame = i * frames_per_process
        end_frame = (i + 1) * frames_per_process if i != num_processes - 1 else total_frames
        p = Process(target=process_frames, args=(start_frame, end_frame, lock, output_dir, start_time, frame_times, total_frames, i))
        processes.append(p)
        p.start()

    # 等待所有进程完成
    for p in processes:
        p.join()

    images_to_video(fps=30)
    print("所有帧处理完毕。")